In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
dataset = '/content/drive/MyDrive/infrance training/model/keypoint_classifier/keypoint.csv'
model_save_path = '/content/drive/MyDrive/infrance training/model/keypoint_classifier/weights.hdf5'
tflite_save_path = '/content/drive/MyDrive/infrance training/model/keypoint_classifier/model.tflite'

In [5]:
NUM_CLASSES = 4

In [6]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [7]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 4)                 44        
                                                                 
Total params: 1,114
Trainable params: 1,114
Non-trainable params: 0
_________________________________________________________________


In [11]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [12]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
22/29 [=====================>........] - ETA: 0s - loss: 1.3065 - accuracy: 0.3523  
Epoch 1: saving model to /content/drive/MyDrive/infrance training/model/keypoint_classifier/weights.hdf5
29/29 [==============================] - 3s 18ms/step - loss: 1.2969 - accuracy: 0.3599 - val_loss: 1.1945 - val_accuracy: 0.4378
Epoch 2/1000
17/29 [================>.............] - ETA: 0s - loss: 1.2095 - accuracy: 0.4062
Epoch 2: saving model to /content/drive/MyDrive/infrance training/model/keypoint_classifier/weights.hdf5
29/29 [==============================] - 0s 5ms/step - loss: 1.2132 - accuracy: 0.3953 - val_loss: 1.1005 - val_accuracy: 0.5238
Epoch 3/1000
24/29 [=======================>......] - ETA: 0s - loss: 1.1628 - accuracy: 0.4271
Epoch 3: saving model to /content/drive/MyDrive/infrance training/model/keypoint_classifier/weights.hdf5
29/29 [==============================] - 0s 5ms/step - loss: 1.1528 - accuracy: 0.4368 - val_loss: 1.0217 - val_accuracy: 0.5890
Epoch 4

In [14]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 [==============================] - 0s 2ms/step - loss: 0.1960 - accuracy: 0.9599


In [15]:
model = tf.keras.models.load_model(model_save_path)

In [16]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[9.6414655e-01 3.5688311e-02 1.3938524e-04 2.5803067e-05]
0


In [17]:
model.save(model_save_path, include_optimizer=False)

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /tmp/tmpkd5yu893/assets


6416